In [1]:
#Libraries used
import pandas as pd #for the data frame
import numpy as np #used for filtering and some arithmatic. 

In [2]:
#Import the data
df_import = pd.read_csv('data/pbp/raw/pbp_query_20162017.csv',low_memory=False) 

In [3]:
#The first thing to do is reduce the number of columns then rows that won't be used
df = df_import[['season', 'game_id', 'game_date', 'event_index', 'game_period',
                'game_seconds', 'event_type', 'event_description', 'event_detail',
                'event_team', 'coords_x', 'coords_y', 'home_team', 'away_team',
                'home_score', 'away_score', 'game_strength_state', 'event_distance',
                'event_angle']]
even_strength = df.game_strength_state.isin(['5v5','4v4','3v3'])
play_type     = df.event_type.isin(['FAC','SHOT','HIT','BLOCK','MISS','GIVE','TAKE','GOAL'])
no_coords     = ~df.coords_x.isna()
df            = df[even_strength & play_type & no_coords]

In [170]:
#Flipping the x/y coordinates for away team in 1st/3rd period and Home 2nd period.
#This makes sure that everything orients the same way. 
away_flip_1 = (df.event_team == df.away_team) & (df.game_period.isin([1,3]))
home_flip_1 = (df.event_team == df.home_team) & (df.game_period.isin([2,4]))
home_cond = (df.logat == 1)
away_flip_0 = (df.event_team == df.away_team) & (df.game_period.isin([2,4]))
home_flip_0 = (df.event_team == df.home_team) & (df.game_period.isin([1,3]))
away_cond = (df.logat == 0)
#Come back to this
df['x_flip'] = np.where((away_flip_1 | home_flip_1) & home_cond ,-1*df.coords_x,df.coords_x)
df['y_flip'] = np.where((away_flip_1 | home_flip_1) & home_cond ,-1*df.coords_y,df.coords_y)
df['x_flip'] = np.where((away_flip_0 | home_flip_0) & away_cond ,df.x_flip,-1*df.coords_x)
df['y_flip'] = np.where((away_flip_0 | home_flip_0) & away_cond ,df.y_flip,-1*df.coords_y)
#This moves the 0,0 point from center ice to the center of the goal on the red line
df['x_goaline'] = 89 - df.x_flip
#How to get the distance
df['new_distance'] = ((df['x_goaline']**2) + (df['y_flip']**2))**(1/2)

In [171]:
df['logat'] = np.where(df.game_id.isin(get_lst),1,0)

In [175]:
df_1.sample(5)

,event_team,home_team,game_period,game_seconds,logat,coords_x,coords_y,x_flip,y_flip,x_goaline,event_distance,new_distance,event_type,event_description
182954,T.B,PIT,2,1214,1,45.0,-34.0,-45.0,34.0,134.0,55.61,138.246157,SHOT,"T.B ONGOAL - #22 CONDRA, Wrist, Off. Zone, 56 ft."
91740,PHI,PHI,2,1704,0,54.0,1.0,-54.0,-1.0,143.0,35.01,143.003496,SHOT,"PHI ONGOAL - #9 PROVOROV, Snap, Off. Zone, 36 ft."
302567,FLA,FLA,3,3564,0,-51.0,20.0,-51.0,20.0,140.0,42.94,141.421356,SHOT,"FLA ONGOAL - #36 JOKINEN, Backhand, Off. Zone,..."
242412,VAN,NSH,3,3383,1,-35.0,7.0,35.0,-7.0,54.0,54.45,54.451814,SHOT,"VAN ONGOAL - #27 HUTTON, Slap, Off. Zone, 55 ft."
112600,ANA,EDM,1,42,1,-19.0,-36.0,19.0,36.0,70.0,78.71,78.714675,SHOT,"ANA ONGOAL - #7 COGLIANO, Wrist, Neu. Zone, 79..."


In [172]:
check_lst = ['event_team','home_team','game_period','game_seconds','logat',
             'coords_x','coords_y','x_flip','y_flip','x_goaline',
             'event_distance','new_distance','event_type','event_description']
df_rc = df[check_lst].copy()
#df_rc['x_wtf'] = 89 - df.x_flip
#df_rc['new_distance_2'] = ((df_rc['x_wtf']**2) + (df_rc['y_flip']**2))**(1/2)

In [174]:
df_rc.sample(10)

,event_team,home_team,game_period,game_seconds,logat,coords_x,coords_y,x_flip,y_flip,x_goaline,event_distance,new_distance,event_type,event_description
195154,DET,DET,1,238,0,-93.0,36.0,-93.0,36.0,182.0,36.22,185.526278,HIT,"DET #39 MANTHA HIT PIT #4 SCHULTZ, Off. Zone"
78384,MTL,CAR,2,2280,0,-85.0,7.0,-85.0,7.0,174.0,8.06,174.140748,BLOCK,MTL #11 GALLAGHER BLOCKED BY CAR #16 LINDHOLM...
25110,BOS,BOS,3,3393,1,44.0,15.0,-44.0,-15.0,133.0,47.43,133.843192,MISS,"BOS #46 KREJCI, Wrist, Wide of Net, Off. Zone,..."
197746,PHI,WSH,2,1954,0,-59.0,-16.0,-59.0,-16.0,148.0,34.00,148.862353,SHOT,"PHI ONGOAL - #15 DEL ZOTTO, Wrist, Off. Zone, ..."
179206,ARI,ANA,3,2592,1,0.0,0.0,-0.0,-0.0,89.0,89.00,89.000000,FAC,ARI won Neu. Zone - ARI #86 JOORIS vs ANA #48 ...
37309,FLA,DET,1,167,0,-69.0,22.0,69.0,-22.0,20.0,29.73,29.732137,FAC,FLA won Def. Zone - FLA #21 TROCHECK vs DET #5...
194370,COL,COL,3,2431,0,-69.0,22.0,-69.0,22.0,158.0,29.73,159.524293,FAC,COL won Off. Zone - NSH #12 FISHER vs COL #34 ...
80766,EDM,DAL,2,1404,0,-52.0,5.0,-52.0,5.0,141.0,37.34,141.088625,BLOCK,"EDM #98 PULJUJARVI BLOCKED BY DAL #47 ODUYA, ..."
160378,TOR,FLA,2,1819,0,-91.0,36.0,-91.0,36.0,180.0,36.06,183.564702,HIT,"TOR #16 MARNER HIT FLA #55 DEMERS, Off. Zone"
316010,OTT,OTT,1,270,1,59.0,-7.0,-59.0,7.0,148.0,30.81,148.165448,BLOCK,"OTT #5 CECI BLOCKED BY CHI #38 HARTMAN, Slap,..."


In [173]:
df_1 = df_rc[df_rc.event_type.isin(['SHOT'])]

In [160]:
df_1.sample(10) 

,event_team,home_team,game_period,game_seconds,logat,coords_x,coords_y,x_flip,y_flip,x_goaline,event_distance,new_distance,event_type,event_description
234363,PHI,PHI,2,1432,0,63.0,25.0,63.0,25.0,26.0,36.07,36.069378,SHOT,"PHI ONGOAL - #93 VORACEK, Wrist, Off. Zone, 36..."
198364,N.J,VAN,3,2902,0,49.0,8.0,49.0,8.0,40.0,40.79,40.792156,SHOT,"N.J ONGOAL - #13 CAMMALLERI, Wrist, Off. Zone,..."
102412,FLA,CHI,3,2888,0,75.0,-12.0,75.0,-12.0,14.0,18.44,18.439089,SHOT,"FLA ONGOAL - #13 PYSYK, Wrist, Off. Zone, 19 ft."
6939,CBJ,CBJ,2,2180,1,-63.0,-38.0,63.0,38.0,26.0,46.04,46.043458,SHOT,"CBJ ONGOAL - #10 WENNBERG, Wrist, Off. Zone, 4..."
307140,MTL,EDM,1,943,1,-49.0,-8.0,49.0,8.0,40.0,40.79,40.792156,SHOT,"MTL ONGOAL - #26 PETRY, Wrist, Off. Zone, 41 ft."
123620,BUF,BUF,3,3303,1,76.0,2.0,-76.0,-2.0,165.0,13.15,165.012121,SHOT,"BUF ONGOAL - #23 REINHART, Wrist, Off. Zone, 1..."
55726,N.J,N.J,3,2630,1,76.0,-17.0,-76.0,17.0,165.0,21.40,165.873446,SHOT,"N.J ONGOAL - #51 KALININ, Backhand, Off. Zone,..."
352426,PHI,PHI,3,2534,0,-61.0,24.0,61.0,-24.0,28.0,36.88,36.878178,SHOT,"PHI ONGOAL - #17 SIMMONDS, Wrist, Off. Zone, 3..."
293530,CAR,ARI,1,51,1,-34.0,36.0,34.0,-36.0,55.0,65.73,65.734314,SHOT,"CAR ONGOAL - #22 PESCE, Wrist, Off. Zone, 66 ft."
226133,MTL,MTL,1,538,0,-61.0,28.0,61.0,-28.0,28.0,39.60,39.597980,SHOT,"MTL ONGOAL - #65 SHAW, Wrist, Off. Zone, 40 ft."


In [135]:
df_1.x_flip.describe()

count    55616.000000
mean        30.700500
std         54.753906
min        -97.000000
25%          8.000000
50%         52.000000
75%         72.000000
max         98.000000
Name: x_flip, dtype: float64

In [71]:
hometeam = df.event_team == df.home_team
shots    = df.event_type == 'SHOT'
first_period = df.game_period == 1
df_1 = df[hometeam & shots & first_period]

In [89]:
game_lst = []
team_lst = []
mean_lst = []
for game in df_1.game_id.unique():
    df_temp = df_1[df_1.game_id==game]
    game_lst.append(game)
    team_lst.append(df_temp.home_team.iloc[0])
    mean_lst.append(df_temp.coords_x.mean())
    
new_df = pd.DataFrame(list(zip(game_lst,team_lst,mean_lst)),
                      columns=['Game','Team','avg'])

In [113]:
testing = new_df[new_df.avg > 0].Game.tolist()

In [111]:
get_lst = new_df_1.Game.tolist()

In [176]:
testing

[2016020001,
 2016020003,
 2016020004,
 2016020005,
 2016020007,
 2016020008,
 2016020009,
 2016020011,
 2016020014,
 2016020015,
 2016020018,
 2016020019,
 2016020021,
 2016020023,
 2016020025,
 2016020028,
 2016020030,
 2016020031,
 2016020034,
 2016020037,
 2016020038,
 2016020041,
 2016020042,
 2016020043,
 2016020047,
 2016020051,
 2016020053,
 2016020055,
 2016020060,
 2016020062,
 2016020063,
 2016020065,
 2016020066,
 2016020069,
 2016020073,
 2016020077,
 2016020078,
 2016020080,
 2016020083,
 2016020084,
 2016020086,
 2016020089,
 2016020093,
 2016020094,
 2016020096,
 2016020097,
 2016020098,
 2016020100,
 2016020103,
 2016020106,
 2016020107,
 2016020112,
 2016020113,
 2016020116,
 2016020118,
 2016020120,
 2016020122,
 2016020126,
 2016020129,
 2016020131,
 2016020133,
 2016020135,
 2016020141,
 2016020145,
 2016020146,
 2016020147,
 2016020150,
 2016020151,
 2016020155,
 2016020157,
 2016020158,
 2016020160,
 2016020162,
 2016020165,
 2016020166,
 2016020167,
 2016020169,